# Bayesian Optimization


In [1]:
!pip install hpsklearn

## Load data

In [1]:
import numpy as np
import pandas as pd
data_aia = pd.read_csv('AIA_allfeature.csv')
data_train = pd.read_csv('AIA_train.csv')
data_test = pd.read_csv('AIA_test.csv')

origin_cate_feature = ['service_type', 'complaint_level', 'contract_type', 'gender', 'is_mix_service',
                       'is_promise_low_consume',
                       'many_over_bill', 'net_service']

origin_num_feature = ['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee',
                      'age', 'contract_time',
                      'former_complaint_fee', 'former_complaint_num',
                      'last_month_traffic', 'local_caller_time', 'local_trafffic_month', 'month_traffic',
                      'online_time', 'pay_num', 'pay_times', 'service1_caller_time', 'service2_caller_time']
feature = data_aia.columns[:-2]
data_aia[origin_cate_feature] = data_aia[origin_cate_feature].astype('category')
data_train[origin_cate_feature] = data_train[origin_cate_feature].astype('category')
data_test[origin_cate_feature] = data_test[origin_cate_feature].astype('category')

In [2]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

data_train['label']=le.fit_transform(data_train['label'].values)
data_train

,service_type,complaint_level,contract_type,gender,is_mix_service,is_promise_low_consume,many_over_bill,net_service,1_total_fee,2_total_fee,...,4_total_feeW1,4_total_feeW2,4_total_feeW3,4_total_feeW4,4_total_feeW5,4_total_feeW6,4_total_feeW7,4_total_feeW8,4_total_feeW9,label
0,4,0,3,2,0,0,0,4,78.18,56.0,...,1.926587,3.013871,-0.752324,1.272943,1.895517,1.827070,-0.852203,-1.497426,-2.037968,1
1,1,0,0,2,0,0,1,4,86.60,52.8,...,1.999853,3.047367,-0.361521,0.384707,1.156852,-1.060000,1.632733,-3.290275,1.833175,0
2,1,0,0,2,0,0,1,4,43.80,54.1,...,2.310467,3.029250,-0.423323,0.249131,1.706402,-0.829686,1.532641,-2.907971,1.440202,4
3,1,0,0,1,0,0,1,4,73.50,57.9,...,-0.422283,2.799761,-2.208144,0.369489,2.037206,1.006464,0.410433,-2.871569,0.674421,4
4,1,0,0,1,0,0,1,4,74.00,74.8,...,0.171605,-0.095337,-0.001438,-0.208894,0.079483,0.242796,-0.143514,-0.651352,-0.096339,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767520,1,0,0,2,0,0,1,4,24.70,34.6,...,-0.064877,1.604699,-0.634678,0.524983,0.293704,-0.723036,0.589189,-4.733323,2.983706,4
767521,1,0,0,2,0,0,1,4,52.70,55.5,...,2.022993,2.439579,-0.937087,-0.007577,2.429035,-1.171271,1.070634,-2.542449,0.990967,4
767522,4,0,1,1,0,1,0,4,240.00,240.0,...,3.052548,0.365216,3.289185,-1.867787,-0.353546,0.008393,2.455316,-0.775706,1.715284,10
767523,1,0,0,2,0,0,1,4,39.40,34.1,...,1.775873,1.997064,-1.009133,-0.048375,3.303548,-0.749326,0.001213,-2.243190,1.000716,4


In [3]:
X = data_train[feature]
X

,service_type,complaint_level,contract_type,gender,is_mix_service,is_promise_low_consume,many_over_bill,net_service,1_total_fee,2_total_fee,...,4_total_feeW0,4_total_feeW1,4_total_feeW2,4_total_feeW3,4_total_feeW4,4_total_feeW5,4_total_feeW6,4_total_feeW7,4_total_feeW8,4_total_feeW9
0,4,0,3,2,0,0,0,4,78.18,56.0,...,-1.606659,1.926587,3.013871,-0.752324,1.272943,1.895517,1.827070,-0.852203,-1.497426,-2.037968
1,1,0,0,2,0,0,1,4,86.60,52.8,...,-3.748563,1.999853,3.047367,-0.361521,0.384707,1.156852,-1.060000,1.632733,-3.290275,1.833175
2,1,0,0,2,0,0,1,4,43.80,54.1,...,-3.731409,2.310467,3.029250,-0.423323,0.249131,1.706402,-0.829686,1.532641,-2.907971,1.440202
3,1,0,0,1,0,0,1,4,73.50,57.9,...,-0.025582,-0.422283,2.799761,-2.208144,0.369489,2.037206,1.006464,0.410433,-2.871569,0.674421
4,1,0,0,1,0,0,1,4,74.00,74.8,...,-0.102678,0.171605,-0.095337,-0.001438,-0.208894,0.079483,0.242796,-0.143514,-0.651352,-0.096339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767520,1,0,0,2,0,0,1,4,24.70,34.6,...,-2.938930,-0.064877,1.604699,-0.634678,0.524983,0.293704,-0.723036,0.589189,-4.733323,2.983706
767521,1,0,0,2,0,0,1,4,52.70,55.5,...,-3.429913,2.022993,2.439579,-0.937087,-0.007577,2.429035,-1.171271,1.070634,-2.542449,0.990967
767522,4,0,1,1,0,1,0,4,240.00,240.0,...,3.944912,3.052548,0.365216,3.289185,-1.867787,-0.353546,0.008393,2.455316,-0.775706,1.715284
767523,1,0,0,2,0,0,1,4,39.40,34.1,...,-3.121953,1.775873,1.997064,-1.009133,-0.048375,3.303548,-0.749326,0.001213,-2.243190,1.000716


In [4]:
y = data_train['label']
y

0          1
1          0
2          4
3          4
4          4
          ..
767520     4
767521     4
767522    10
767523     4
767524     2
Name: label, Length: 767525, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)
x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=100)

In [10]:
import lightgbm as lgb

train_data = lgb.Dataset(data=x_train,label=y_train)
test_data = lgb.Dataset(data=x_test,label=y_test)

## custom hyperopt's space for optimizing

In [11]:
from hyperopt import fmin, tpe, hp, partial

space = {"max_depth": hp.randint("max_depth", 15),
         "num_trees": hp.randint("num_trees", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "bagging_fraction": hp.randint("bagging_fraction", 5),
         "num_leaves": hp.randint("num_leaves", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['num_trees'] = argsDict['num_trees'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["bagging_fraction"] = argsDict["bagging_fraction"] * 0.1 + 0.5
    argsDict["num_leaves"] = argsDict["num_leaves"] * 3 + 10
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

## Initiate the model

In [12]:
from sklearn.metrics import mean_squared_error

def lightgbm_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)

    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'num_trees': argsDict['num_trees'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'bagging_fraction': argsDict['bagging_fraction'],  # 采样数
              'num_leaves': argsDict['num_leaves'],  # 终点节点最小样本占比的和
              'objective': 'multiclass',
              'feature_fraction': 0.7,  # 样本列采样
              'lambda_l1': 0,  # L1 正则化
              'lambda_l2': 0,  # L2 正则化
              'bagging_seed': 100,  # 随机种子,light中默认为100
              }
    params['metric'] = ['multiclass']
    params['num_class'] = [12]

    model_lgb = lgb.train(params, train_data, num_boost_round=50, valid_sets=[test_data],early_stopping_rounds=100)

    return get_tranformer_score(model_lgb)

def get_tranformer_score(tranformer):

    model = tranformer
    prediction = model.predict(x_predict, num_iteration=model.best_iteration)
    prediction = np.argmax(prediction, axis=1)
    return mean_squared_error(y_predict, prediction)

## start hyper parameters optimizing

In [13]:
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(lightgbm_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

  0%|                                                                           | 0/20 [00:00<?, ?trial/s, best loss=?]

C:\Users\peiwa\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[LightGBM] [Warning]                                                                                                   
Auto-choosing col-wise multi-threading, the overhead of testing was 0.233058 seconds.                                  
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                                                                                      
Total Bins 23131                                                                                                       
[LightGBM] [Info]                                                                                                      
Number of data points in the train set: 552617, number of used features: 110                                           
  0%|                                                                           | 0/20 [00:03<?, ?trial/s, best loss=?]

C:\Users\peiwa\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')

C:\Users\peiwa\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))



[LightGBM] [Info]                                                                                                      
Start training from score -2.076336                                                                                    
[LightGBM] [Info]                                                                                                      
Start training from score -2.083679                                                                                    
[LightGBM] [Info]                                                                                                      
Start training from score -2.417634                                                                                    
[LightGBM] [Info]                                                                                                      
Start training from score -3.442457                                                                                    
[LightGBM] [Info]                       

...

[LightGBM] [Warning]                                                                                                   
Auto-choosing col-wise multi-threading, the overhead of testing was 0.489386 seconds.                                  
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                                                                                      
Total Bins 23131                                                                                                       
[LightGBM] [Info]                                                                                                      
Number of data points in the train set: 552617, number of used features: 110                                           
[LightGBM] [Info]                                                                                                      
Start training from score -2.076336                                                                   

[105]	valid_0's multi_logloss: 0.521521                                                                                
[106]	valid_0's multi_logloss: 0.5212                                                                                  
[107]	valid_0's multi_logloss: 0.520872                                                                                
[108]	valid_0's multi_logloss: 0.520531                                                                                
[109]	valid_0's multi_logloss: 0.52019                                                                                 
[110]	valid_0's multi_logloss: 0.519857                                                                                
[111]	valid_0's multi_logloss: 0.519559                                                                                
[112]	valid_0's multi_logloss: 0.519237                                                                                
[113]	valid_0's multi_logloss: 0.518901 

C:\Users\peiwa\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[LightGBM] [Warning]                                                                                                   
Auto-choosing col-wise multi-threading, the overhead of testing was 0.516021 seconds.                                  
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]                                                                                                      
Total Bins 23131                                                                                                       
[LightGBM] [Info]                                                                                                      
Number of data points in the train set: 552617, number of used features: 110                                           
[LightGBM] [Info]                                                                                                      
Start training from score -2.076336                                                                   

[105]	valid_0's multi_logloss: 0.535498                                                                                
[106]	valid_0's multi_logloss: 0.53502                                                                                 
[107]	valid_0's multi_logloss: 0.534659                                                                                
[108]	valid_0's multi_logloss: 0.534246                                                                                
[109]	valid_0's multi_logloss: 0.533804                                                                                
[110]	valid_0's multi_logloss: 0.533382                                                                                
[111]	valid_0's multi_logloss: 0.533                                                                                   
[112]	valid_0's multi_logloss: 0.532582                                                                                
[113]	valid_0's multi_logloss: 0.532168 

In [14]:
logloss = lightgbm_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('multi_logloss of the best lightgbm:', np.sqrt(logloss))

C:\Users\peiwa\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.274296 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23131
[LightGBM] [Info] Number of data points in the train set: 552617, number of used features: 110
[LightGBM] [Info] Start training from score -2.076336
[LightGBM] [Info] Start training from score -2.083679
[LightGBM] [Info] Start training from score -2.417634
[LightGBM] [Info] Start training from score -3.442457
[LightGBM] [Info] Start training from score -0.931308
[LightGBM] [Info] Start training from score -3.123901
[LightGBM] [Info] Start training from score -3.586747
[LightGBM] [Info] Start training from score -3.883511
[LightGBM] [Info] Start training from score -3.509125
[LightGBM] [Info] Start training from score -3.440197
[LightGBM] [Info] Start training from score -2.896316
[LightGBM] [Info] Start training from score -3.668136
[1]	valid_0's multi_logloss: 1.69215
Training until valid

[185]	valid_0's multi_logloss: 0.507198
[186]	valid_0's multi_logloss: 0.50712
[187]	valid_0's multi_logloss: 0.507003
[188]	valid_0's multi_logloss: 0.506873
[189]	valid_0's multi_logloss: 0.506766
[190]	valid_0's multi_logloss: 0.50668
[191]	valid_0's multi_logloss: 0.506575
[192]	valid_0's multi_logloss: 0.506494
[193]	valid_0's multi_logloss: 0.50643
[194]	valid_0's multi_logloss: 0.50633
[195]	valid_0's multi_logloss: 0.506256
[196]	valid_0's multi_logloss: 0.506157
[197]	valid_0's multi_logloss: 0.506037
[198]	valid_0's multi_logloss: 0.505955
[199]	valid_0's multi_logloss: 0.505866
[200]	valid_0's multi_logloss: 0.505811
[201]	valid_0's multi_logloss: 0.50572
[202]	valid_0's multi_logloss: 0.505649
[203]	valid_0's multi_logloss: 0.50554
[204]	valid_0's multi_logloss: 0.505451
[205]	valid_0's multi_logloss: 0.505356
[206]	valid_0's multi_logloss: 0.505239
[207]	valid_0's multi_logloss: 0.505169
[208]	valid_0's multi_logloss: 0.505087
[209]	valid_0's multi_logloss: 0.504998
[210]	

[391]	valid_0's multi_logloss: 0.495678
[392]	valid_0's multi_logloss: 0.495646
[393]	valid_0's multi_logloss: 0.495619
[394]	valid_0's multi_logloss: 0.495575
[395]	valid_0's multi_logloss: 0.495505
[396]	valid_0's multi_logloss: 0.495467
Did not meet early stopping. Best iteration is:
[396]	valid_0's multi_logloss: 0.495467
best : {'bagging_fraction': 0.8, 'learning_rate': 0.05157882542320766, 'max_depth': 14, 'num_leaves': 25, 'num_trees': 396}
best param after transform :
{'bagging_fraction': 0.5800000000000001, 'learning_rate': 0.051031576508464155, 'max_depth': 19, 'num_leaves': 85, 'num_trees': 546}
rmse of the best lightgbm: 2.265410788850312


## Results

In [17]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

lgb_model = lgb.LGBMClassifier(
    boosting_type="gbdt", objective='multiclass', class_weight='balanced',learning_rate= 0.05, 
        max_depth= 14, bagging_fraction= 0.8, num_trees= 396, 
                    num_leaves=25
)

lgb_model.fit(x_train, y_train)

C:\Users\peiwa\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] num_iterations is set=396, num_trees=396 will be ignored. Current value: num_iterations=396


LGBMClassifier(bagging_fraction=0.8, class_weight='balanced',
               learning_rate=0.05, max_depth=14, num_leaves=25, num_trees=396,
               objective='multiclass')

In [18]:
from sklearn.metrics import f1_score
f1_score(y_true=y_test, y_pred=lgb_model.predict(x_test), average='macro')

0.7716182662949461

## Gridsearch

In [ ]:
lgb_model = lgb.LGBMClassifier(
    boosting_type="gbdt", objective='multiclass', class_weight='balanced')

clf = GridSearchCV(
    estimator = lgb_model,
    param_grid = {'num_leaves':[8,10,12],
                  'learning_rate': [0.05,0.1,0.5,1] , 
                  'max_depth': [7,8,9],
                  'subsample':[0.55, 0.65, 0.75]},
#     param_grid = {
#                   'learning_rate': [0.04, 0.05, 0.06, 0.07] ,            
#                   },
    cv=StratifiedKFold(n_splits=5).split(data_train[feature], data_train.label)
)
clf.fit(x_train, y_train)

In [ ]:
clf.best_params_

In [19]:
# best_params

lgb_model = lgb.LGBMClassifier(
    boosting_type="gbdt", objective='multiclass', class_weight='balanced',learning_rate= 0.07, 
        max_depth= 9, subsample= 0.55, num_leaves=12
)

lgb_model.fit(x_train, y_train)

LGBMClassifier(class_weight='balanced', learning_rate=0.07, max_depth=9,
               num_leaves=12, objective='multiclass', subsample=0.55)

In [20]:
from sklearn.metrics import f1_score
f1_score(y_true=y_test, y_pred=lgb_model.predict(x_test), average='macro')

0.7467202934651761

## Default Hyper Parameters

In [7]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
lgb_model = lgb.LGBMClassifier(
    boosting_type="gbdt", objective='multiclass', class_weight='balanced')

lgb_model.fit(x_train, y_train)

LGBMClassifier(class_weight='balanced', objective='multiclass')

In [9]:
lgb_model

LGBMClassifier(class_weight='balanced', objective='multiclass')

In [8]:
from sklearn.metrics import f1_score
f1_score(y_true=y_test, y_pred=lgb_model.predict(x_test), average='macro')

0.7667528243176714

## F1-scores:
Default: 0.767 <p>
Bayesian Opt: 0.772 <p>
Grid-search: 0.746

Conclusion: Bayesian OPT has better performance than default hyper parameters 